## pba-modul-5-build-ner-tagger-using-sklearncrfsuite

In [1]:
!pip install scikit-learn==0.24.0

  Using cached scikit-learn-0.24.0.tar.gz (7.4 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for scikit-learn (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [1416 lines of output]
      <string>:17: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
      Partial import of sklearn during the build process.
      <string>:116: DeprecationWarning:
      
        `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
        of the deprecation of `distutils` itself. It will be removed for
        Python >= 3.12. For older Python versions it will remain present.
        It is recommended to use `setuptools < 60.0` for those Python versions.
        For more details, see:
          https://numpy.org/devdocs/reference/distutils_status_migration.ht

In [10]:
import pandas as pd

df=pd.read_csv('ner_dataset.csv',encoding='ISO-8859-1')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   Sentence #  47959 non-null    object
 1   Word        1048565 non-null  object
 2   POS         1048575 non-null  object
 3   Tag         1048575 non-null  object
dtypes: object(4)
memory usage: 32.0+ MB


In [11]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,1048565,1048566,1048567,1048568,1048569,1048570,1048571,1048572,1048573,1048574
Sentence #,Sentence: 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Sentence: 47959,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Word,Thousands,of,demonstrators,have,marched,through,London,to,protest,the,...,impact,.,Indian,forces,said,they,responded,to,the,attack
POS,NNS,IN,NNS,VBP,VBN,IN,NNP,TO,VB,DT,...,NN,.,JJ,NNS,VBD,PRP,VBD,TO,DT,NN
Tag,O,O,O,O,O,O,B-geo,O,O,O,...,O,O,B-gpe,O,O,O,O,O,O,O


In [12]:
df = df.fillna(method='ffill')
df.info()

/tmp/ipykernel_8972/1354937737.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   Sentence #  1048575 non-null  object
 1   Word        1048575 non-null  object
 2   POS         1048575 non-null  object
 3   Tag         1048575 non-null  object
dtypes: object(4)
memory usage: 32.0+ MB


In [14]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,1048565,1048566,1048567,1048568,1048569,1048570,1048571,1048572,1048573,1048574
Sentence #,Sentence: 1,Sentence: 1,Sentence: 1,Sentence: 1,Sentence: 1,Sentence: 1,Sentence: 1,Sentence: 1,Sentence: 1,Sentence: 1,...,Sentence: 47958,Sentence: 47958,Sentence: 47959,Sentence: 47959,Sentence: 47959,Sentence: 47959,Sentence: 47959,Sentence: 47959,Sentence: 47959,Sentence: 47959
Word,Thousands,of,demonstrators,have,marched,through,London,to,protest,the,...,impact,.,Indian,forces,said,they,responded,to,the,attack
POS,NNS,IN,NNS,VBP,VBN,IN,NNP,TO,VB,DT,...,NN,.,JJ,NNS,VBD,PRP,VBD,TO,DT,NN
Tag,O,O,O,O,O,O,B-geo,O,O,O,...,O,O,B-gpe,O,O,O,O,O,O,O


In [15]:
df['Sentence #'].nunique(), df.Word.nunique(), df.POS.nunique(), df.Tag.nunique()

(47959, 35177, 42, 17)

1.2 Tag Distribution

In [16]:
df.Tag.value_counts()

Tag
O        887908
B-geo     37644
B-tim     20333
B-org     20143
I-per     17251
B-per     16990
I-org     16784
B-gpe     15870
I-geo      7414
I-tim      6528
B-art       402
B-eve       308
I-art       297
I-eve       253
B-nat       201
I-gpe       198
I-nat        51
Name: count, dtype: int64

In [17]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }

    if i > 0:
        word1, postag1 = sent[i - 1][:2]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent) - 1:
        word1, postag1 = sent[i + 1][:2]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]


def sent2labels(sent):
    return [label for _, _, label in sent]


In [18]:
agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(),
                                                  s['POS'].values.tolist(),
                                                  s['Tag'].values.tolist())]


In [19]:
grouped_df = df.groupby('Sentence #').apply(agg_func)

In [20]:
print(grouped_df[grouped_df.index == 'Sentence: 1'].values)

[list([('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')])]


In [21]:
grouped_df.shape

(47959,)

In [22]:
sentences = [s for s in grouped_df]
sentences[0]

[('Thousands', 'NNS', 'O'),
 ('of', 'IN', 'O'),
 ('demonstrators', 'NNS', 'O'),
 ('have', 'VBP', 'O'),
 ('marched', 'VBN', 'O'),
 ('through', 'IN', 'O'),
 ('London', 'NNP', 'B-geo'),
 ('to', 'TO', 'O'),
 ('protest', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('war', 'NN', 'O'),
 ('in', 'IN', 'O'),
 ('Iraq', 'NNP', 'B-geo'),
 ('and', 'CC', 'O'),
 ('demand', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('withdrawal', 'NN', 'O'),
 ('of', 'IN', 'O'),
 ('British', 'JJ', 'B-gpe'),
 ('troops', 'NNS', 'O'),
 ('from', 'IN', 'O'),
 ('that', 'DT', 'O'),
 ('country', 'NN', 'O'),
 ('.', '.', 'O')]

In [23]:
sent2features(sentences[0][5:7])

[{'bias': 1.0,
  'word.lower()': 'through',
  'word[-3:]': 'ugh',
  'word[-2:]': 'gh',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'IN',
  'postag[:2]': 'IN',
  'BOS': True,
  '+1:word.lower()': 'london',
  '+1:word.istitle()': True,
  '+1:word.isupper()': False,
  '+1:postag': 'NNP',
  '+1:postag[:2]': 'NN'},
 {'bias': 1.0,
  'word.lower()': 'london',
  'word[-3:]': 'don',
  'word[-2:]': 'on',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'postag': 'NNP',
  'postag[:2]': 'NN',
  '-1:word.lower()': 'through',
  '-1:word.istitle()': False,
  '-1:word.isupper()': False,
  '-1:postag': 'IN',
  '-1:postag[:2]': 'IN',
  'EOS': True}]

In [24]:
sent2labels(sentences[0][5:7])

['O', 'B-geo']

In [25]:
import numpy as np
from sklearn.model_selection import train_test_split

X = np.asarray([sent2features(s) for s in sentences], dtype="object")
y = np.asarray([sent2labels(s) for s in sentences], dtype="object")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


Building Models with sklearn-crfsuite

In [26]:
!pip install git+https://github.com/MeMartijn/updated-sklearn-crfsuite.git

  Cloning https://github.com/MeMartijn/updated-sklearn-crfsuite.git to /tmp/pip-req-build-udfpw6wz
  Running command git clone --filter=blob:none --quiet https://github.com/MeMartijn/updated-sklearn-crfsuite.git /tmp/pip-req-build-udfpw6wz
  Resolved https://github.com/MeMartijn/updated-sklearn-crfsuite.git to commit 675038761b4405f04691a83339d04903790e2b95
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.6 MB/s eta 0:00:00m eta 0:00:010:00:01
  Created wheel for sklearn-crfsuite: filename=sklearn_crfsuite-0.3.6-py2.py3-none-any.whl size=10868 sha256=44020b99f212561c41d72b3a80207afa70a32cf925d7851b23f272ad2398630a
  Stored in directory: /tmp/pip-ephem-wheel-cache-b9s4bsov/wheels/4f/7f/ba/bab38c044a215c1428211e718eab6a8808c87545d4ae3985bf
Successfully built sklearn-crfsuite


3 Train the model!

In [27]:
import sklearn_crfsuite

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
    verbose=True
)


In [28]:
crf.fit(X_train, y_train)

loading training data to CRFsuite: 100%|██████████| 35969/35969 [00:09<00:00, 3960.15it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 133629
Seconds required: 1.822

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=2.68  loss=1264028.18 active=132637 feature_norm=1.00
Iter 2   time=2.70  loss=994060.37 active=131294 feature_norm=4.42
Iter 3   time=1.33  loss=776414.63 active=125970 feature_norm=3.87
Iter 4   time=6.76  loss=422140.67 active=127018 feature_norm=3.24
Iter 5   time=1.31  loss=355771.79 active=129029 feature_norm=4.04
Iter 6   time=1.36  loss=264115.87 active=124043 feature_norm=6.10
Iter 7   time=1.34  loss=222293.53 active=117180 feature_norm=7.69
Iter 8   time=1.38  loss=197820.35 active=110837 feature_norm=8.75
Iter 9   time=1.32  loss=176879.78 active=105646 feature_norm

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    max_iterations=100, verbose=True)

In [29]:
import joblib
joblib.dump(crf, 'ner_model.pkl')

['ner_model.pkl']

In [30]:
crf = joblib.load('ner_model.pkl')

In [31]:
y_pred = crf.predict(X_test)
print(y_pred[0])

['O', 'O', 'O', 'O', 'B-per', 'I-per', 'O', 'B-org', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [32]:
print(y_test[0])

['O', 'O', 'O', 'O', 'B-per', 'I-per', 'O', 'B-org', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [33]:
from sklearn_crfsuite import metrics as crf_metrics
labels = list(crf.classes_)
labels.remove('O')

In [34]:
print(crf_metrics.flat_classification_report(y_test, y_pred, labels=labels))

              precision    recall  f1-score   support

       B-org       0.81      0.73      0.77      5116
       B-per       0.85      0.84      0.84      4239
       I-per       0.85      0.90      0.88      4273
       B-geo       0.86      0.92      0.89      9403
       I-geo       0.81      0.80      0.81      1826
       B-tim       0.93      0.89      0.91      5095
       I-org       0.82      0.79      0.81      4195
       B-gpe       0.97      0.94      0.96      3961
       I-tim       0.84      0.81      0.82      1604
       B-nat       0.65      0.24      0.35        55
       B-eve       0.49      0.34      0.40        80
       B-art       0.38      0.15      0.21       102
       I-art       0.24      0.07      0.10        90
       I-eve       0.41      0.20      0.27        74
       I-gpe       0.86      0.53      0.66        36
       I-nat       0.67      0.22      0.33        18

   micro avg       0.86      0.85      0.86     40167
   macro avg       0.72   

4.1 End-to-End NER Tagger with trained NER Model

In [35]:
import re

text = """Three more countries have joined an “international grand committee”
of parliaments, adding to calls for
Facebook’s boss, Mark Zuckerberg, to give evidence on misinformation to the
coalition. Brazil, Latvia and Singapore
bring the total to eight different parliaments across the world, with plans to
send representatives to London on 27
November with the intention of hearing from Zuckerberg. Since the Cambridge
Analytica scandal broke, the Facebook chief
has only appeared in front of two legislatures: the American Senate and House
of Representatives, and the European parliament.
Facebook has consistently rebuffed attempts from others, including the UK and
Canadian parliaments, to hear from Zuckerberg.
He added that an article in the New York Times on Thursday, in which the paper
alleged a pattern of behaviour from Facebook
to “delay, deny and deflect” negative news stories, “raises further questions
about how recent data breaches were allegedly
dealt with within Facebook."""
text = re.sub(r'\n', '', text)
print(text)


Three more countries have joined an “international grand committee”of parliaments, adding to calls forFacebook’s boss, Mark Zuckerberg, to give evidence on misinformation to thecoalition. Brazil, Latvia and Singaporebring the total to eight different parliaments across the world, with plans tosend representatives to London on 27November with the intention of hearing from Zuckerberg. Since the CambridgeAnalytica scandal broke, the Facebook chiefhas only appeared in front of two legislatures: the American Senate and Houseof Representatives, and the European parliament.Facebook has consistently rebuffed attempts from others, including the UK andCanadian parliaments, to hear from Zuckerberg.He added that an article in the New York Times on Thursday, in which the paperalleged a pattern of behaviour from Facebookto “delay, deny and deflect” negative news stories, “raises further questionsabout how recent data breaches were allegedlydealt with within Facebook.


4.1.2 NER Tagging with SpaCy

In [36]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 9.2 MB/s eta 0:00:00m eta 0:00:010:01:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [37]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
text_nlp = nlp(text)
displacy.render(text_nlp, style='ent', jupyter=True)

4.1.3 Pipeline Step 1

In [38]:
import nltk
text_tokens = nltk.word_tokenize(text)
text_pos = nltk.pos_tag(text_tokens)
text_pos[:10]

[('Three', 'CD'),
 ('more', 'JJR'),
 ('countries', 'NNS'),
 ('have', 'VBP'),
 ('joined', 'VBN'),
 ('an', 'DT'),
 ('“', 'NNP'),
 ('international', 'JJ'),
 ('grand', 'JJ'),
 ('committee', 'NN')]

4.1.4 Pipeline Step 2

In [39]:
features = [sent2features(text_pos)]
features[0][0]

{'bias': 1.0,
 'word.lower()': 'three',
 'word[-3:]': 'ree',
 'word[-2:]': 'ee',
 'word.isupper()': False,
 'word.istitle()': True,
 'word.isdigit()': False,
 'postag': 'CD',
 'postag[:2]': 'CD',
 'BOS': True,
 '+1:word.lower()': 'more',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'JJR',
 '+1:postag[:2]': 'JJ'}

4.1.5 Pipeline Step 3

In [40]:
labels = crf.predict(features)
doc_labels = labels[0]
doc_labels[10:20]

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

4.1.6 Pipeline Step 4

In [41]:
text_ner = [(token, tag) for token, tag in zip(text_tokens, doc_labels)]
print(text_ner)

[('Three', 'O'), ('more', 'O'), ('countries', 'O'), ('have', 'O'), ('joined', 'O'), ('an', 'O'), ('“', 'O'), ('international', 'O'), ('grand', 'O'), ('committee', 'O'), ('”', 'O'), ('of', 'O'), ('parliaments', 'O'), (',', 'O'), ('adding', 'O'), ('to', 'O'), ('calls', 'O'), ('forFacebook', 'O'), ('’', 'O'), ('s', 'O'), ('boss', 'O'), (',', 'O'), ('Mark', 'B-per'), ('Zuckerberg', 'I-per'), (',', 'O'), ('to', 'O'), ('give', 'O'), ('evidence', 'O'), ('on', 'O'), ('misinformation', 'O'), ('to', 'O'), ('thecoalition', 'O'), ('.', 'O'), ('Brazil', 'B-geo'), (',', 'O'), ('Latvia', 'B-geo'), ('and', 'O'), ('Singaporebring', 'O'), ('the', 'O'), ('total', 'O'), ('to', 'O'), ('eight', 'O'), ('different', 'O'), ('parliaments', 'O'), ('across', 'O'), ('the', 'O'), ('world', 'O'), (',', 'O'), ('with', 'O'), ('plans', 'O'), ('tosend', 'O'), ('representatives', 'O'), ('to', 'O'), ('London', 'B-geo'), ('on', 'O'), ('27November', 'B-tim'), ('with', 'O'), ('the', 'O'), ('intention', 'O'), ('of', 'O'), ('h

In [42]:
named_entities = []
temp_entity_name = ''
temp_named_entity = None

for term, tag in text_ner:
    if tag != 'O':
        temp_entity_name = ' '.join([temp_entity_name, term]).strip()
        temp_named_entity = (temp_entity_name, tag)
    else:
        if temp_named_entity:
            named_entities.append(temp_named_entity)
        temp_entity_name = ''
        temp_named_entity = None

# If there is a named entity at the end of the text
if temp_named_entity:
    named_entities.append(temp_named_entity)


In [43]:
import pandas as pd
pd.DataFrame(named_entities, columns=['Entity', 'Tag'])

,Entity,Tag
0,Mark Zuckerberg,I-per
1,Brazil,B-geo
2,Latvia,B-geo
3,London,B-geo
4,27November,B-tim
5,Zuckerberg,B-geo
6,Facebook,B-art
7,American Senate and Houseof Representatives,I-org
8,UK,B-org
9,Zuckerberg.He,B-geo
